In [23]:
import pandas as pd
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import mean_squared_error

file_name = "Heart_Disease_Classification_Dataset.csv"
data = pd.read_csv(file_name)
data['sex'] = data['sex'].map({'male': 1, 'female': 0})
data['target'] = data['target'].map({'yes': 1, 'no': 0})
print("\nValue Mappings:")
print("Sex Mapping: {'male': 1, 'female': 0}")
print("Target Mapping: {'yes': 1, 'no': 0}")



Value Mappings:
Sex Mapping: {'male': 1, 'female': 0}
Target Mapping: {'yes': 1, 'no': 0}


In [19]:
def report_columns(df):
    print("Column Report:")
    print("-" * 30)
    for col in df.columns:
        col_type = df[col].dtype
        print(f"Column Name: {col}")
        print(f"Data Type: {col_type}")
        print("-" * 30)
report_columns(data)

Column Report:
------------------------------
Column Name: Unnamed: 0
Data Type: int64
------------------------------
Column Name: age
Data Type: int64
------------------------------
Column Name: sex
Data Type: int64
------------------------------
Column Name: cp
Data Type: int64
------------------------------
Column Name: trestbps
Data Type: float64
------------------------------
Column Name: chol
Data Type: float64
------------------------------
Column Name: fbs
Data Type: int64
------------------------------
Column Name: restecg
Data Type: int64
------------------------------
Column Name: thalach
Data Type: float64
------------------------------
Column Name: exang
Data Type: int64
------------------------------
Column Name: oldpeak
Data Type: float64
------------------------------
Column Name: slope
Data Type: int64
------------------------------
Column Name: ca
Data Type: int64
------------------------------
Column Name: thal
Data Type: int64
------------------------------
Column N

In [16]:
print("Dataset Preview:")
print(data.head())

Dataset Preview:
   Unnamed: 0  age  sex  cp  trestbps   chol  fbs  restecg  thalach  exang  \
0           0   63    1   3     145.0  233.0    1        0    150.0      0   
1           1   37    1   2     130.0  250.0    0        1    187.0      0   
2           2   41    0   1     130.0  204.0    0        0    172.0      0   
3           3   56    1   1     120.0  236.0    0        1    178.0      0   
4           4   57    0   0       NaN  354.0    0        1    163.0      1   

   oldpeak  slope  ca  thal  target  
0      2.3      0   0     1       1  
1      3.5      0   0     2       1  
2      1.4      2   0     2       1  
3      0.8      2   0     2       1  
4      0.6      2   0     2       1  


In [18]:
print("\nBasic Report:")
print(data.describe())


Basic Report:
       Unnamed: 0         age         sex          cp    trestbps        chol  \
count  303.000000  303.000000  303.000000  303.000000  299.000000  302.000000   
mean   151.000000   54.366337    0.683168    0.966997  131.712375  246.317881   
std     87.612784    9.082101    0.466011    1.032052   17.629032   51.908285   
min      0.000000   29.000000    0.000000    0.000000   94.000000  126.000000   
25%     75.500000   47.500000    0.000000    0.000000  120.000000  211.000000   
50%    151.000000   55.000000    1.000000    1.000000  130.000000  240.500000   
75%    226.500000   61.000000    1.000000    2.000000  140.000000  274.750000   
max    302.000000   77.000000    1.000000    3.000000  200.000000  564.000000   

              fbs     restecg     thalach       exang     oldpeak       slope  \
count  303.000000  303.000000  298.000000  303.000000  303.000000  303.000000   
mean     0.148515    0.528053  149.865772    0.326733    1.039604    1.399340   
std      0.3

In [25]:
print("\nMissing Values by Column:")
missing_values = data.isnull().sum()
print(missing_values[missing_values > 0])


Missing Values by Column:
Series([], dtype: int64)


In [24]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder


def fill_missing_values(df):
    missing_values = df.isnull().sum()
    missing_columns = missing_values[missing_values > 0].index.tolist()
    report = []
    for col in missing_columns:
        if df[col].dtype in ['float64', 'int64']:
            if df[col].isna().sum() == len(df[col]):
                # pass  columns that are entirely NaN
                continue
            
            unique_values = df[col].dropna().unique()
            if len(unique_values) == 2:
                # Column is binary
                continue
            missing_before = df[col].isna().sum()
            
            not_nan = df[df[col].notna()]
            nan = df[df[col].isna()]

            X_train = not_nan.drop(columns=[col])
            y_train = not_nan[col]

            X_train = pd.get_dummies(X_train, drop_first=True)
            X_train = X_train.fillna(0)

            model = LinearRegression()
            model.fit(X_train, y_train)

            X_nan = nan.drop(columns=[col])
            X_nan = pd.get_dummies(X_nan, drop_first=True)
            X_nan = X_nan.fillna(0)

            df.loc[nan.index, col] = model.predict(X_nan)

            missing_after = df[col].isna().sum()
            
            #  report updating
            report.append({
                'Column Name': col,
                'Missing Before': missing_before,
                'Missing After': missing_after
            })
        
        else:
            mode_imputer = SimpleImputer(strategy='most_frequent')
            df[col] = mode_imputer.fit_transform(df[[col]])
            
            missing_before = df[col].isna().sum()
            missing_after = df[col].isna().sum()
            
            report.append({
                'Column Name': col,
                'Missing Before': missing_before,
                'Missing After': missing_after
            })

    return df, report
df_filled, report = fill_missing_values(data)
print("DataFrame with Missing Values Filled:")
print(df_filled)
print("\nMissing Values Report:")
print("-" * 40)
for entry in report:
    print(f"Column Name: {entry['Column Name']}")
    print(f"Missing Before: {entry['Missing Before']}")
    print(f"Missing After: {entry['Missing After']}")
    print("-" * 40)
    
df_filled.to_csv(file_name.replace(".","_Cleaned."))


DataFrame with Missing Values Filled:
     Unnamed: 0  age  sex  cp    trestbps   chol  fbs  restecg  thalach  \
0             0   63    1   3  145.000000  233.0    1        0    150.0   
1             1   37    1   2  130.000000  250.0    0        1    187.0   
2             2   41    0   1  130.000000  204.0    0        0    172.0   
3             3   56    1   1  120.000000  236.0    0        1    178.0   
4             4   57    0   0  131.894925  354.0    0        1    163.0   
..          ...  ...  ...  ..         ...    ...  ...      ...      ...   
298         298   57    0   0  140.000000  241.0    0        1    123.0   
299         299   45    1   3  110.000000  264.0    0        1    132.0   
300         300   68    1   0  144.000000  193.0    1        1    141.0   
301         301   57    1   0  127.934644  131.0    0        1    115.0   
302         302   57    0   1  130.000000  236.0    0        0    174.0   

     exang  oldpeak  slope  ca  thal  target  
0        0    